# Key Processing

In [1]:
import pandas as pd
import glob
import re
import os.path
import numpy as np
from openpyxl import load_workbook



In [2]:
databases = ['Asset Suite', 'Archibus', 'COBRA','EDMS','FIS','IPS','P6','PID','Site People','Time Billing']
path = 'C:\\Users\\\mounsl\\OneDrive - Idaho National Laboratory\\Documents - GRP-Agree Data Assessment Team\\Data Assessment Taxonomy\\PK-FK files\\' #path to where files for the database are kept

## Key Processing

In [16]:
dd_gloss = pd.read_excel('Data Elements Taxonomy_copy.xlsx', sheet_name='Data Dictionary')
writer = pd.ExcelWriter('Data Elements Taxonomy_PK_FK.xlsx', engine='openpyxl', mode = 'a', if_sheet_exists='overlay')
clean_PKs = pd.DataFrame()
clean_FKs = pd.DataFrame()
for database in databases:    
    #PKs
    if os.path.isfile(path+database+'-PKs.xlsx'):
        # print(database)
        raw_PKs = pd.read_excel(path+database+'-PKs.xlsx')
        raw_PKs = raw_PKs.rename(columns={'TableName':'TABLENAME','ColumnName':'COLNAME','ORDINAL_POSITION':'Primary Key'}).drop(columns=['SchemaName','PK_name'])
        raw_PKs['System'] = database
        raw_PKs['TABLENAME'] = raw_PKs['TABLENAME'].str.strip()
        raw_PKs['COLNAME'] = raw_PKs['COLNAME'].str.strip()
        raw_PKs = raw_PKs[['System','TABLENAME','COLNAME','Primary Key']]
        clean_PKs = pd.concat([clean_PKs,raw_PKs])

    #FKS
    if os.path.isfile(path+database+'-FKs.xlsx'):
        raw_FKs = pd.read_excel(path+database+'-FKs.xlsx')
        raw_FKs = raw_FKs.rename(columns={'TableName':'TABLENAME','ColumnName':'COLNAME'}).drop(columns=['SchemaName','FK_name'])
        raw_FKs['System'] = database
        raw_FKs['TABLENAME'] = raw_FKs['TABLENAME'].str.strip()
        raw_FKs['COLNAME'] = raw_FKs['COLNAME'].str.strip()
        clean_FKs = pd.concat([clean_FKs,raw_FKs])

dd_gloss['System'] = dd_gloss['System'].str.strip()
dd_gloss['TABLENAME'] = dd_gloss['TABLENAME'].str.strip()
dd_gloss['COLNAME'] = dd_gloss['COLNAME'].str.strip()
dd_gloss_pk = dd_gloss.merge(clean_PKs, on=['System','TABLENAME','COLNAME'], how='left')
dd_gloss_pk_fk = dd_gloss_pk.merge(clean_FKs, on=['System','TABLENAME','COLNAME'], how='left')
## DROP DUPLICATES?
# fk_dd = dd_gloss.merge(clean_FKs, on=['System','TABLENAME','COLNAME'], how='right')

In [17]:
# #dd_gloss_pk.to_excel('Data Elements Taxonomy_PK.xlsx', index=False)
# #fk_dd[['System','TABLENAME','COLNAME','ReferencedTable','ReferencedColumn']].to_excel('Data Elements Foreign Keys.xlsx',index=False)
dd_gloss_pk_fk.to_excel(writer, sheet_name = 'Data Dictionary', index=False)
writer.close()


# Asset Suite Key "Query"

In [6]:
# as_path = 'c:\\Users\\mounsl\\OneDrive - Idaho National Laboratory\\Documents - GRP-Agree Data Assessment Team\\Asset Suite\\Working documents - Start here\\'
# files = glob.glob(as_path + 'DataDictionary*.xlsx')
# as_pks = pd.DataFrame(columns=['Dictionary','TableName','ColumnName','ORDINAL_POSITION'])
# table_count = 0
# for file in files:
#     workbook = pd.read_excel(file, sheet_name=None, keep_default_na = False)
#     for key, df in workbook.items():
#         if '-' in key:
#             table_count += 1
#             tableName = re.sub(' ', '',key).split('-')[-1]
#             # print(tableName)
#             index_rows = df[(df.iloc[:,0].str.contains('INDEX', na=False))&(df.iloc[:,0].str.contains('CREATE', na=False))].index
            
#             for rownum in index_rows:
#                 pks = ''
#                 i=rownum+1
#                 while df.iloc[i,0]!='':
#                     pks = pks+re.sub('[() ]', '', df.iloc[i,0])
#                     i += 1
#                 # print(pks[0:len(pks)]) 
#                 pk_list = pks.split(',')
#                 j = 1
#                 for pk in pk_list:
#                     as_pks.loc[len(as_pks)] = {'Dictionary':file.split('_',1)[1], 'TableName':tableName,'ColumnName':pk,'ORDINAL_POSITION':j}
#                     j+=1
# as_pks['SchemaName'] = ''
# as_pks['PK_name'] = 'PK'
# as_pks = as_pks.drop_duplicates()


In [7]:
# as_pks.to_excel(path+'Asset Suite-PKs.xlsx', index=False)

In [8]:
## XXX Manually decide which create index statement to use on the following tables:
## TIDOBLOB, TIDTPCOL, TIDECPHY, TIDLBSTP, TIDPMFUT

# P6 Key "Query"

In [9]:
# p6_path = 'C:\\Users\\mounsl\\OneDrive - Idaho National Laboratory\Documents - GRP-Agree Data Assessment Team\\Project Management\\3. Data and modeling\\Data Dictionary\\P6\\PXRPTUSER Scripted Views\\'
# p6files = glob.glob(p6_path + '*vw.txt')


# table_pattern = re.compile(r'(?<=C:\\Users\\mounsl\\OneDrive - Idaho National Laboratory\\Documents - GRP-Agree Data Assessment Team\\Project Management\\3\. Data and modeling\\Data Dictionary\\P6\\PXRPTUSER Scripted Views\\).*(?=vw\.txt)')
# pk_pattern = re.compile(r'(?<=[ \.])objectid')  ## TODO: this is selecting the a.name_id portion, need to select the alias and UPPERCASE it
# fk_pattern = re.compile(r'(?<=[/. ])\w+(?=objectid)')
# p6_pks = pd.DataFrame(columns=['SchemaName','TableName','ColumnName','PK_name','ORDINAL_POSITION'])
# p6_fks = pd.DataFrame(columns=['SchemaName','TableName','ColumnName','FK_name','ReferencedTable','ReferencedColumn'])
# for file in p6files:
#     with open(file, 'r') as fp:
#         tableName = table_pattern.search(file)[0]
#         # print(tableName)
#     # read all lines using readline()
#         lines = fp.readlines()
#         j=1
#         for row in lines:
#             # check if string present on a current line
#             if pk_pattern.search(row):
#                 # pk_name = pk_pattern.search(row)[0].upper()
#                 # print('PK for '+tableName+': '+ pk_name)
#                 p6_pks.loc[len(p6_pks)] = {'TableName':tableName,'ColumnName':'OBJECTID','ORDINAL_POSITION':j}
#                 j +=1
#             if fk_pattern.search(row):
#                 fk_table = fk_pattern.search(row)[0].upper()
#                 # print('FK for '+tableName+': '+ fk_table)
#                 p6_fks.loc[len(p6_fks)] = {'TableName':tableName,'ColumnName':fk_table+'OBJECTID','ReferencedTable':fk_table,'ReferencedColumn':'OBJECTID'}


In [10]:
# p6_pks.to_excel(path+'P6-PKs.xlsx', index=False)
# p6_fks.to_excel(path+'P6-FKs.xlsx', index=False)

In [11]:
# p6 = dd_gloss[dd_gloss['System']=='P6']#['TABLENAME'].nunique()

In [12]:
# no_objectid = p6.groupby('TABLENAME').filter(lambda x: ~(x['COLNAME']=='OBJECTID').any())
# no_objectid['TABLENAME'].unique()

In [13]:
# p6_fk_val = p6[(p6['COLNAME'].str.contains('OBJECTID'))&(p6['COLNAME']!='OBJECTID')].merge(p6_fks, left_on=['TABLENAME','COLNAME'],right_on=['TableName','ColumnName'], how = 'outer')[['TABLENAME','COLNAME','ReferencedTable','ReferencedColumn']]

In [14]:
## Likely missed FKs
# p6_fk_val[p6_fk_val['ReferencedTable'].isna()]

In [15]:
## FK references a table not in taxonomy
# p6_fks[~p6_fks['ReferencedTable'].isin(p6['TABLENAME'])].head(50)

In [16]:
# p6_val = p6[p6['COLNAME']=='OBJECTID'].merge(p6_pks, left_on=['TABLENAME','COLNAME'],right_on=['TableName','ColumnName'], how = 'outer')[['TABLENAME','COLNAME','TableName','ColumnName']]

In [17]:
## Likely missed PKs
# p6_val[p6_val['TableName'].isna()]

# Validation

In [32]:
# as_pks[as_pks['ORDINAL_POSITION']==1]['TableName'].nunique()

In [37]:
## Tables in the DD
dd_gloss['TABLENAME'].nunique()

306

In [39]:
dd_gloss_pk['TABLENAME'].nunique()

306

In [40]:
## Tables with PK identified
dd_gloss_pk[dd_gloss_pk['Primary Key']==1]['TABLENAME'].nunique()


255

In [41]:
no_pks = dd_gloss_pk.groupby('TABLENAME').filter(lambda x: x['Primary Key'].isnull().all())

In [42]:
no_pks.groupby(['System','TABLENAME']).max('Primary Key')#.to_clipboard()

Cardinality  Max Length  Primary Key
System       TABLENAME                                                     
COBRA        BDNDETL                          21.0         NaN          NaN
             COB_NTX                          10.0         NaN          NaN
EDMS         A_DRAFTERS                        8.0        20.0          NaN
             A_DRAWINGS                    63863.0       800.0          NaN
             CLOSED_DARS                  247706.0        50.0          NaN
             DRAW_NUMBERING               245491.0       100.0          NaN
             DRF                          115002.0      1955.0          NaN
             ENGINEERS                        30.0        23.0          NaN
             ERROR_LOG                        24.0       147.0          NaN
             ESSENTIAL_AREA                 1510.0        15.0          NaN
             ESSENTIAL_LOG                 17053.0        23.0          NaN
             MARKUPTAB                         1.0       567.0          NaN
             OLD_CAD                       76069.0        33.0          NaN
             SPECIAL_CODES                150472.0        17.0          NaN
             SPECIAL_CODES_LOG              8297.0        23.0          NaN
             STW_DRAWINGS                  63131.0        30.0          NaN
             SUPERSEDED_DRAWINGS           15815.0        10.0          NaN
IPS          CNCodeImports                    76.0        27.0          NaN
             datWBS_INL                     6679.0        65.0          NaN
             dat_MRSendClick                   1.0         NaN          NaN
P6           ACTIVITYCODEASSIGNMENT       145386.0       120.0          NaN
             ACTIVITYCODEHEIRARCHY        184726.0       116.0          NaN
             ACTIVITYSPREAD               145386.0       120.0          NaN
             COSTACCOUNTHIERARCHY         183484.0        43.0          NaN
             CURRENCY                      56115.0       120.0          NaN
             EPSHIERARCHY                 183484.0        43.0          NaN
             FINANCIALPERIOD               56115.0       120.0          NaN
             GLOBALPROFILE                 56115.0       120.0          NaN
             GLOBALSECURITY                  235.0         1.0          NaN
             PROJECTCODEASSIGNMENT         56115.0       100.0          NaN
             PROJECTCODEHIERARCHY         184726.0       116.0          NaN
             PROJECTETLLIST                 1252.0         NaN          NaN
             PROJECTPROJECTPORTFOLIO       56115.0        10.0          NaN
             PXSERVICES                    56115.0        98.0          NaN
             PXSETTINGS                    56115.0      3997.0          NaN
             RELATIONSHIP                1754511.0       120.0          NaN
             RESOURCEACCESS                 3088.0        63.0          NaN
             RESOURCEASSIGNMENTSPREAD     145386.0         NaN          NaN
             RESOURCECODEASSIGNMENT        56115.0        63.0          NaN
             RESOURCECODEHIERARCHY        184726.0       116.0          NaN
             RESOURCEHIERARCHY            183484.0        43.0          NaN
             RESOURCERESOURCETEAM          56115.0        10.0          NaN
             UDFTYPE                       56115.0        70.0          NaN
             UDFVALUE                     217874.0       255.0          NaN
             USERMODULEACCESS              56115.0        15.0          NaN
             USERPROFILEPRIVILEGE          56115.0        76.0          NaN
             WBSHEIRARCHY                 183484.0        43.0          NaN
Site People  SitePeople                    11452.0        50.0          NaN
             ntusers                       12216.0        42.0          NaN
Time Billing PS_IF_BALANCES               163185.0         6.0          NaN

In [45]:
## Duplicate rows
dd_gloss_pk[dd_gloss_pk.duplicated(subset=['Org Order'])].sort_values('Org Order')['TABLENAME']


1094    TIDLBSTP
1096    TIDLBSTP
1098    TIDLBSTP
1101    TIDLBSTP
1514    TIDECPHY
Name: TABLENAME, dtype: object